## Response Generation

In [1]:
from vllm import LLM, SamplingParams
from datasets import load_dataset
import pandas as pd 
from math_verify import parse, verify
from collections import defaultdict

def most_frequent_sublist(data):
    """
    Return (sublist, count) for the most frequent sublist in `data`.
    Ties are broken by the earliest first appearance.
    """
    if not data:
        return None, 0

    counts = defaultdict(int)
    first_idx = {}

    for i, item in enumerate(data):
        t = tuple(item)              # lists are unhashable; use tuples as keys
        counts[t] += 1
        if t not in first_idx:
            first_idx[t] = i

    # max by (count, -first_idx) → highest count, earliest position wins ties
    best = max(counts, key=lambda t: (counts[t], -first_idx[t]))
    return list(best), counts[best]
    
# 모델 불러오기
llm = LLM(
    model='./qwen2.5-0.5b-instruct-q4_0.gguf',
    tokenizer="Qwen/Qwen2.5-0.5B-Instruct",
    trust_remote_code=True,
    dtype="auto",
    max_model_len=2048,
    gpu_memory_utilization=0.6,
)

INFO 10-13 01:48:38 [__init__.py:216] Automatically detected platform cuda.
INFO 10-13 01:48:39 [utils.py:328] non-default args: {'tokenizer': 'Qwen/Qwen2.5-0.5B-Instruct', 'trust_remote_code': True, 'max_model_len': 2048, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'model': './qwen2.5-0.5b-instruct-q4_0.gguf'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-13 01:49:09 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


ERROR 10-13 01:49:09 [config.py:278] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './qwen2.5-0.5b-instruct-q4_0.gguf'., retrying 1 of 2
ERROR 10-13 01:49:11 [config.py:276] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './qwen2.5-0.5b-instruct-q4_0.gguf'.
INFO 10-13 01:49:11 [__init__.py:2764] Downcasting torch.float32 to torch.bfloat16.
INFO 10-13 01:49:11 [__init__.py:1815] Using max model len 2048
WARNING 10-13 01:49:11 [__init__.py:1217] gguf quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 10-13 01:49:27 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-13 01:49:28 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorc

[W1013 01:49:45.211156951 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=14698) INFO 10-13 01:49:46 [gpu_model_runner.py:2370] Loading model from scratch...
(EngineCore_DP0 pid=14698) INFO 10-13 01:49:58 [cuda.py:362] Using Flash Attention backend on V1 engine.
(EngineCore_DP0 pid=14698) INFO 10-13 01:50:06 [gpu_model_runner.py:2392] Model loading took 0.3987 GiB and 19.529778 seconds
(EngineCore_DP0 pid=14698) INFO 10-13 01:50:09 [backends.py:539] Using cache directory: /home/ssafy/.cache/vllm/torch_c

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████████████████████| 67/67 [00:09<00:00,  7.29it/s]


(EngineCore_DP0 pid=14698) INFO 10-13 01:50:27 [gpu_model_runner.py:3118] Graph capturing finished in 10 secs, took 0.50 GiB
(EngineCore_DP0 pid=14698) INFO 10-13 01:50:27 [gpu_worker.py:391] Free memory on device (3.2/4.0 GiB) on startup. Desired GPU memory utilization is (0.6, 2.4 GiB). Actual usage is 0.4 GiB for weight, 1.39 GiB for peak activation, 0.0 GiB for non-torch memory, and 0.5 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=-35922740` to fit into requested memory, or `--kv-cache-memory=824329012` to fully utilize gpu memory. Current kv cache memory in use is 654040268 bytes.
(EngineCore_DP0 pid=14698) INFO 10-13 01:50:27 [core.py:218] init engine (profile, create kv cache, warmup model) took 21.37 seconds
INFO 10-13 01:50:28 [llm.py:295] Supported_tasks: ['generate']
INFO 10-13 01:50:28 [__init__.py:36] No IOProcessor plugins requested by the model


In [3]:
# 프롬프트 불러오기

df = load_dataset('HAERAE-HUB/KoSimpleEval', "hrm_gsm", split='test').to_pandas().head(20)

prompts = []

for _, row in df.iterrows():

    prompts.append([
        {"role": "system", "content":"You are a helpful Korean assistant."},
        {"role": "user", "content": row['question']+"\n최종 답을 \\boxed{N}의 형태로 적어주세요."}
    ])

## No Test-Time Scaling

In [25]:
# 답변 생성
sampling = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=2048,
     repetition_penalty=1.01
)
outputs = llm.chat(prompts, sampling)
df["response"] = [o.outputs[0].text for o in outputs]

Adding requests:   0%|          | 0/20 [00:00<?, ?it/s]

Processed prompts:   0%|                     | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

In [26]:
output = []
for _,row in df.iterrows():
    try:
        response = row.response#.split('</think>')[1]
        resp = parse(response)
        
        is_correct0 = verify(parse(row.gold),resp)
        is_correct1 = row.gold == resp[-1]

        output.append(
            any([is_correct0,is_correct1])
        )
    except:
        output.append(False)
df['correct'] = output     

overall_acc = df["correct"].mean()
print(f"Overall accuracy: {overall_acc:.2%}")

Overall accuracy: 0.00%


## Majority Voting

In [22]:
N = 32
sampling = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=2048,
     repetition_penalty=1.01,
    n=N
)
outputs = llm.chat(prompts, sampling)

Adding requests:   0%|          | 0/20 [00:00<?, ?it/s]

Processed prompts:   0%|                    | 0/640 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

In [23]:
output = []
for o,(_,row) in zip(outputs,df.iterrows()):
    try:
        resps = []
        for i in range(N):
            resp = parse(o.outputs[i].text)
            resps.append(resp)
        sublist, count = most_frequent_sublist(resps)
    
        is_correct0 = verify(parse(row.gold),sublist)
        is_correct1 = row.gold == sublist[-1]
    
        output.append(
                any([is_correct0,is_correct1])
            )
    except:
        output.append(False)
df['correct'] = output    

ValueError: Length of values (0) does not match length of index (20)

In [8]:
# N-32
overall_acc = df["correct"].mean()
print(f"Overall accuracy: {overall_acc:.2%}")

Overall accuracy: 0.00%


In [9]:
df

,question,gold,response,correct
0,Janet의 오리는 하루에 16개의 알을 낳습니다. 그녀는 매일 아침으로 3개를 먹...,18.0,Janet는 매일 농산물 시장에서 신선한 오리 알 하나당 2달러를 판매합니다.,False
1,로브 하나를 만드는데 2개의 파란 섬유와 그 절반의 하얀 섬유가 필요합니다. 총 몇...,3.0,2개의 파란 섬유와 그 절반의 하얀 섬유가 필요합니다. 따라서 총 몇 개의 섬유가 ...,False
2,"Josh는 집을 리모델링하기로 결정합니다. 그는 $80,000에 집을 사고 $50...",70000.0,"Josh의 이익은 $150,000입니다.",False
3,James는 매주 3번씩 3개의 스프린트를 합니다. 그는 각 스프린트마다 60미터를...,540.0,"이 문제를 해결하기 위해서는, 스프린트를 몇 번이나 축소하여 총 몇 미터를 달리는지...",False
4,"Wendi는 하루에 닭 한 마리당 세 컵의 혼합 먹이를 줍니다. 아침에 15컵, 오...",20.0,"Wendi는 아침에 15컵, 오후에 25컵의 혼합 먹이를 줍니다. 마지막 식사로 닭...",False
5,Kylar는 그의 새 아파트를 위해 유리컵을 사기 위해 상점에 갔다. 유리컵 하나의...,64.0,"Kylar는 16개의 유리컵을 구매해야 하는데, 두 번째 유리컵은 가격의 60%를 ...",False
6,"Seattle에 양이 20마리 있다면, Toulouse, Charleston, Se...",260.0,4 마리,False
7,Carla는 200GB 파일을 다운로드 중입니다. 보통 1분에 2GB를 다운로드할 ...,160.0,20분,False
8,"John은 60mph의 속도로 3시간 동안 운전한 후, 집에 아주 중요한 물건을 두...",45.0,"60mph로 3시간동안의 속도로 운전한 후, 집에 60 * 3 = 180km를 향해...",False
9,Eliza가 일주일에 처음 40시간 동안 일할 때의 시간당 임금은 $10입니다. 그...,460.0,"Eliza는 45시간 일했기 때문에, 초과 근무 수당을 적용한 수익은 45 * 1....",False


In [28]:
import json
from pathlib import Path

path = Path("mclm_k2b_32.jsonl")  # change to your file
records = []

with path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:      # skip blank lines
            continue
        records.append(json.loads(line))

In [29]:
from math_verify import parse, verify
import random
from tqdm import tqdm
scores = []
for n in tqdm([2,5,6,8,10,12,14,16,18,20,22,24,26,28,30,32]):
    output = []
    for row in records:
        try:
            resps = random.sample(row['responses'],n)
            resps = [parse(resp) for resp in resps]
            sublist, count = most_frequent_sublist(resps)
            
            is_correct0 = verify(parse(row['gold']),sublist)
            is_correct1 = row['gold'] == sublist[-1]
        
            output.append(
                    any([is_correct0,is_correct1])
                )
        except:
            output.append(False)
    score = sum(output)/len(records)
    scores.append(score)

  0%|                                                                                            | 0/16 [00:00<?, ?it/s]Timeout during parsing: 주어진 복소수 $ z $와 $ c $의 위상(위치) 변화에 대해 설명하겠습니다.

---

### 주어진 정보

- $ z = 2 + \sqrt{2} - (3 + 3\sqrt{2})i $
- $ c = 2 - 3i $

- $ w $는 $ z $가 $ c $를 중심으로 반시계 방향으로 $ \frac{\pi}{4} $만큼 회전한 결과입니다.

---

### 복소수와 각도

복소수 $ z $와 $ c $의 실수부와 허수부를 각각 구하면,  
- $ z = 2 + \sqrt{2} - (3 + 3\sqrt{2})i $  
- $ c = 2 - 3i $

이렇게 나타낼 수 있습니다.

#### 복소수 각도(위상)

복소수 $ z $와 $ c $의 각도(위상)는 다음과 같이 구할 수 있습니다.

- 복소수 $ z = a + bi $의 위상(각도) $ \theta_z = \tan^{-1}\left(\frac{b}{a}\right) $
- 복소수 $ c = a + bi $의 위상(각도) $ \theta_c = \tan^{-1}\left(\frac{b}{a}\right) $

이렇게 하면 $ z $의 위상은  
$$
\theta_z = \tan^{-1}\left(\frac{- (3 + 3\sqrt{2})}{2 + \sqrt{2}}\right)
$$

$ c $의 위상은  
$$
\theta_c = \tan^{-1}\left(\frac{-3}{2}\right)
$$

---

### 회전

- $ w $는 $ z $를 중심 $ c $로 반시계 방향으로 $ \frac{\pi}{4} $ 만큼 회전한 값입니다.
- 복소수 회전은 원점에서 중심으로 회전하는 것과 달리, 중심점을 기준으로 각도를 더하거나 뺍니다.

복소수 회전 공식

In [30]:
import matplotlib.pyplot as plt
plt.plot(
    [2,5,6,8,10,12,14,16,18,20,22,24,26,28,30,32],
    scores
)

ModuleNotFoundError: No module named 'matplotlib'

## Math-RM

In [32]:
import json
from pathlib import Path

path = Path("mclm_k2b_32.jsonl")  # change to your file
records = []

with path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:      # skip blank lines
            continue
        records.append(json.loads(line))

In [2]:
# from vllm import LLM, SamplingParams
# import numpy as np
# llm = LLM("internlm/internlm2-1_8b-reward", task="classify", trust_remote_code=True)
# tokenizer = llm.get_tokenizer()

INFO 10-08 08:19:41 [__init__.py:216] Automatically detected platform cuda.
INFO 10-08 08:19:43 [utils.py:233] non-default args: {'task': 'classify', 'trust_remote_code': True, 'disable_log_stats': True, 'model': 'internlm/internlm2-1_8b-reward'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-08 08:19:44 [model.py:547] Resolved architecture: InternLM2ForRewardModel


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-08 08:19:44 [model.py:1510] Using max model len 32768
INFO 10-08 08:19:44 [arg_utils.py:1575] (Disabling) chunked prefill by default
INFO 10-08 08:19:44 [arg_utils.py:1578] (Disabling) prefix caching by default
INFO 10-08 08:19:44 [__init__.py:440] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:46 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:46 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='internlm/internlm2-1_8b-reward', speculative_config=None, tokenizer='internlm/internlm2-1_8b-reward', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_ca

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(EngineCore_DP0 pid=17064) INFO 10-08 08:19:51 [default_loader.py:267] Loading weights took 0.49 seconds
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:51 [gpu_model_runner.py:2653] Model loading took 3.1741 GiB and 1.239984 seconds
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:55 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/fb2678dad7/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:55 [backends.py:559] Dynamo bytecode transform time: 3.39 s
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:56 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.069 s
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:57 [monitor.py:34] torch.compile takes 3.39 s in total
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:57 [gpu_worker.py:298] Available KV cache memory: 65.95 GiB
(EngineCore_DP0 pid=17064) INFO 10-08 08:19:58 [kv_cache_utils.py:1087] GPU KV cache size: 720,320 tokens
(EngineCore_DP0 pid=17064) IN

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:01<00:00, 35.33it/s]


(EngineCore_DP0 pid=17064) INFO 10-08 08:20:00 [gpu_model_runner.py:3480] Graph capturing finished in 2 secs, took 0.48 GiB
(EngineCore_DP0 pid=17064) INFO 10-08 08:20:00 [core.py:210] init engine (profile, create kv cache, warmup model) took 9.04 seconds
(EngineCore_DP0 pid=17064) INFO 10-08 08:20:01 [__init__.py:440] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 10-08 08:20:02 [llm.py:306] Supported_tasks: ['encode']


In [ ]:
# import time
# s = time.time()
# outputs = []
# responses = []
# for row in records:
#     prompt = row['question']
#     scores = []
#     cands = []
#     for resp in row['responses']:
#         chat = [
#             {"role": "system", "content": "Please reason step by step, and check your final answer within \\boxed{}."},
#             {"role": "user", "content": prompt},
#             {"role": "assistant", "content": resp}
#         ]
        
#         conversation_str = tokenizer.apply_chat_template(
#             chat, 
#             tokenize=False, 
#             add_generation_prompt=False
#         )
#         str_len = len(tokenizer.apply_chat_template(
#             chat, 
#             tokenize=True, 
#             add_generation_prompt=False
#         ))
        
#         if str_len>4096:
#             scores.append(-1)
#             continue
#         # cands.append(conversation_str)
#         # (output,) = llm.classify(conversation_str)
#         # score = output.outputs.probs[0]
#         (output,) = llm.encode(conversation_str)
#         score = output.outputs.data[-1].item()
#         scores.append(score)
#     idx = np.argmax(scores)
#     selected = row['responses'][idx]
#     responses.append(selected)

# e = time.time()

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# from datasets import load_dataset
# from math_verify import parse, verify

# df = load_dataset('HAERAE-HUB/KoSimpleEval','MCLM', split='test').to_pandas()
# df['response'] = responses

# output = []
# for _,row in df.iterrows():
#     try:
#         response = row.response#.split('</think>')[1]
#         resp = parse(response)
        
#         is_correct0 = verify(parse(row.gold),resp)
#         is_correct1 = row.gold == resp[-1]
    
#         output.append(
#             any([is_correct0,is_correct1])
#         )
#     except:
#         output.append(False)
# df['correct'] = output     

# overall_acc = df["correct"].mean()
# print(f"Overall accuracy: {overall_acc:.2%}")

# # 6) Accuracy by category
# acc_by_cat = df.groupby("category")["correct"].mean().sort_values(ascending=False)
# print("\nAccuracy by category:")
# print(acc_by_cat)

In [47]:
import os

os.environ['OPENROUTER_API_KEY'] = "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"

from litellm import completion

prompt = """You will be given a question and a response.
Return the score of the response from a scale of 1 to 5.
Provide a brief explanation first that return the score in [[N]] format.

Your answer should be formatted in 

{{explanation}}

The score is [[N]]

format. 

Here is the question and response
Question : 
{question}

Response : 
{response}"""

for resp in records[0]['responses']:
    question = records[0]['question']
    query = prompt.format(question=question,response=resp)
    response = completion(
        model= "openrouter/meta-llama/llama-3.3-8b-instruct:free",
        messages=[{"role": "user", "content": query}]
    )
    response = response.choices[0].message.content
    break

In [48]:
import re
def extract_int(s):
    m = re.search(r"\[\[\s*([+-]?\d+)\s*\]\]",s)
    return int(m.group(1)) if m else None

In [49]:
extract_int(response)

5